In [90]:
import numpy as np

In [91]:
#Function using the log-sum-exp trick#
def logSumExp(a):
    b = np.max(a)
    return(b + np.log(np.sum(np.exp(a-b))))

#####################
##Forward Algorithm##
#####################

#Function to run forward algorithm, arguments are n = # obs, m = # states for z,#
#k = # states for x, pi = initial distribution(m vector), 
#Tmat = transition matrix (mxm), phi = emission distribution (m x k matrix)#
#x is the observed data#
#takes log of pi, Tmat and phi

def forwardAlg(n, m, k, pi, Tmat, phi, x):
    g = np.zeros((n,m))
    for i in range(0,m):
        g[0,i] = (pi[i]) + (phi[i, x[0]])
    
    for j in range(1, n):
        for l in range(0, m):
            g[j,l] = logSumExp(g[j-1, :]+(Tmat[:,l])+(phi[l,x[j]-1]))
    return(g)

def pForward(g, x):
    pXf = logSumExp(g[len(x)-1,:])
    return(pXf)

In [92]:
x = np.loadtxt('x.txt')
m = 10
n = len(x)
k = 59
pi = np.full(m, 1/m)
phi = np.full((m,k), 1/k)
Tmat = np.full((m,m), 1/m)

In [93]:
g = forwardAlg(n,m,k,pi,Tmat,phi,x)
pXf = pForward(g,x)
pXf

C:\Users\Megan Robertson\Anaconda3\lib\site-packages\ipykernel\__main__.py:19: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Megan Robertson\Anaconda3\lib\site-packages\ipykernel\__main__.py:23: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


706.50399969663499

In [138]:
def backwardAlg(n, m, k, pi, Tmat, phi, x):
    r = np.zeros((n,m))
    for j in range(n-1, 0, -1):
        for l in range(0, m):
            r[j, l] = logSumExp(r[j, ] + Tmat[l,] + phi[:, x[j]-1])
    
    return(r)

#Function to return p(x_1:n) from matrix from backward algorithm
def pBackward(r, pi, phi, x):
    pXb = logSumExp(r[0, ]+ pi +phi[:,x[0]])
    return(pXb)

In [139]:
r = backwardAlg(n, m, k, pi, Tmat, phi, x)
pBackward(r, pi, phi, x)

C:\Users\Megan Robertson\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Megan Robertson\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


2.4195342455364188

(292, 10)